In [1]:
import os
import tensorflow as tf #tf 2.0.0
import numpy as np

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_data_gen = ImageDataGenerator(rotation_range=50,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    zoom_range=0.3,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    fill_mode='constant',
                                    cval=0,
                                    rescale=1./255)
valid_data_gen = ImageDataGenerator(rotation_range=45,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    zoom_range=0.3,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    fill_mode='constant',
                                    cval=0,
                                    rescale=1./255)

test_data_gen = ImageDataGenerator(rescale=1./255)



In [3]:
dataset_dir = os.path.join('Dataset')

Batch_size = 1
img_h = 112
img_w = 112
num_classes=2

classes = ['m', # 0
            'nm', # 1
           ]

In [4]:
# Training
SEED = 1234
tf.random.set_seed(SEED) 

training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               target_size=(112,112),
                                               batch_size=Batch_size,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)  # targets are directly converted into one-hot vectors

# Validation
valid_dir = os.path.join(dataset_dir, 'valid')
valid_gen = valid_data_gen.flow_from_directory(valid_dir,
                                           target_size=(112,112),
                                           batch_size=Batch_size, 
                                           classes=classes,
                                           class_mode='categorical',
                                           shuffle=False,
                                           seed=SEED)
# Test
test_dir = os.path.join(dataset_dir, 'testing')
test_gen = test_data_gen.flow_from_directory(test_dir,
                                             target_size=(112, 112),
                                             batch_size=1, 
                                             shuffle=False,
                                             seed=SEED,
                                             class_mode=None,
                                             )




Found 3526 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In [5]:
ResNet_model = tf.keras.applications.ResNet152V2(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

In [6]:
from tensorflow.keras import Model 
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential

# The last 15 layers fine tune
for layer in ResNet_model.layers[:-15]:
    layer.trainable = False

x = ResNet_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.3)(x)
output  = Dense(units=2, activation='softmax')(x)
model = Model(ResNet_model.input, output)


model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 118, 118, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 56, 56, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 58, 58, 64)   0           conv1_conv[0][0]                 
______________________________________________________________________________________________

In [7]:
#MobileNet
#rate = 0.4
#mobilenet = tf.keras.applications.MobileNetV2( include_top=False , input_shape=(112,112,3 ), pooling='avg' , weights='imagenet' )
#x = tf.keras.layers.Dense( 512 , activation='relu' )( mobilenet.output )
#x = tf.keras.layers.Dropout( rate )(x)
#x = tf.keras.layers.Dense( 256 , activation='relu' )(x)
#x = tf.keras.layers.Dropout( rate )(x)
#x = tf.keras.layers.Dense( 128 , activation='relu' )(x)
#x = tf.keras.layers.Dropout( rate )(x)
#x = tf.keras.layers.Dense( 64 , activation='relu' )(x)
#x = tf.keras.layers.Dropout( rate )(x)
#x = tf.keras.layers.Dense( 32 , activation='relu' )(x)
#x = tf.keras.layers.Dropout( rate )(x)
#x = tf.keras.layers.Dense( 16 , activation='relu' )(x)
#x = tf.keras.layers.Dropout( rate )(x)
#outputs = tf.keras.layers.Dense( 2 , activation='softmax' )(x)
#MobileNetmodel = tf.keras.Model( mobilenet.input , outputs )
#MobileNetmodel.summary()

In [8]:
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss=loss, metrics= ['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

lrr = ReduceLROnPlateau(monitor='val_accuracy', 
                        patience=3, 
                        verbose=1, 
                        factor=0.4, 
                        min_lr=0.0001)


callbacks = [lrr]

STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size
STEP_SIZE_VALID=valid_gen.n//valid_gen.batch_size
transfer_learning_history = model.fit_generator(generator=train_gen,
                   steps_per_epoch=STEP_SIZE_TRAIN,
                   validation_data=valid_gen,
                   validation_steps=STEP_SIZE_VALID,
                   epochs=100,
                  callbacks=callbacks,
                  class_weight='auto',
                  
                    
)


Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 3526 steps
Epoch 1/100
3526/3526 [==============================] - 107s 30ms/step - loss: 0.6488 - accuracy: 0.6146
Epoch 2/100
3526/3526 [==============================] - 102s 29ms/step - loss: 0.4895 - accuracy: 0.7683
Epoch 3/100
3526/3526 [==============================] - 104s 30ms/step - loss: 0.4371 - accuracy: 0.8086
Epoch 4/100
3526/3526 [==============================] - 103s 29ms/step - loss: 0.4066 - accuracy: 0.8247
Epoch 5/100
3526/3526 [==============================] - 105s 30ms/step - loss: 0.4230 - accuracy: 0.8174
Epoch 6/100
3526/3526 [==============================] - 104s 30ms/step - loss: 0.3938 - accuracy: 0.8364
Epoch 7/100
3526/3526 [==============================] - 103s 29ms/step - loss: 0.3875 - accuracy: 0.8395
Epoch 8/100
3526/3526 [==============================] - 105s 30ms/step - loss: 0.3960 - accuracy: 0.8349
Epoch 9/100
3526/3526 [=======

In [ ]:
model.save('MaskNonMask-classifier.h5')


In [ ]:

#The first step is to import your model using load_model method.
from tensorflow.keras.models import load_model
Maskmodel = load_model('MaskNonMask-classifier.h5')

In [ ]:
Maskmodel

In [ ]:
Maskmodel.compile(optimizer=optimizer, loss=loss, metrics= ['accuracy'])


In [ ]:
from tensorflow.keras.preprocessing import image
test_image1 = image.load_img('3782.jpg', target_size = (112, 112)) 
test_image1 = image.img_to_array(test_image1)
test_image1 = np.expand_dims(test_image1, axis = 0)

#predict the result
result = Maskmodel.predict(test_image1)

In [ ]:
result

In [ ]:
#Saving the trained model
converter = tf.lite.TFLiteConverter.from_keras_model( model )
converter.post_training_quantization = True
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_LATENCY]
buffer = converter.convert()
open( 'Classifier.tflite' , 'wb' ).write( buffer )


### Visualization accuracy and loss 

In [ ]:
import matplotlib.pyplot as plt

acc = transfer_learning_history.history['accuracy']
#val_acc = transfer_learning_history.history['val_accuracy']

loss = transfer_learning_history.history['loss']
#val_loss = transfer_learning_history.history['val_loss']

epochs_range = range(50)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
#plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
#plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

### model evaluate with validation set

In [ ]:
from PIL import Image
image = Image.open('3.jpg')

In [ ]:
image.resize((112,112))
mo

In [ ]:
model.predict(image)